# Task 1: Data Understanding, Preparation and Descriptive Analytics

# Clean the Datasets

## Merge the Datasets

I used the function `merge` from pandas library that implements SQL style joining operations.

In this case, `transactions` is our primary dataset, with each row representing a transaction record. We want to ensure that every transaction is retained in the final merged dataset, even if certain demographic, merchant, or city information is missing.

Using `how='left'` for each merge step ensures **all transactions are retained** in the final dataset, even if:

- **Customer data is missing** Transactions without a matching `cc_num` in `customers` will still appear, with `NaN` for customer details

- **Merchant information is missing** Transactions lacking a matching `merchant` in `merchants` are included, with `NaN` for merchant fields

- **City data is missing** If a customer’s `city` has no match in `cities`, the transaction is kept with `NaN` for city details


In [5]:
import pandas as pd

# Load Datasets
transactions = pd.read_csv('data/transactions.csv')
merchants = pd.read_csv('data/merchants.csv')
customers = pd.read_csv('data/customers.csv')
cities = pd.read_csv('data/cities.csv')

# Merge the .csv files into one
merged_data = pd.merge(transactions, customers, on='cc_num', how='left')
merged_data = pd.merge(merged_data, merchants, on='merchant', how='left')
merged_data = pd.merge(merged_data, cities, on='city', how='left')

# Print merged dataset
print(merged_data.head())

# Save merged datase
merged_data.to_csv('data/merged_data.csv', index=False)

   index trans_date_trans_time            cc_num device_os     merchant  \
0   5381   2023-01-01 00:39:03  2801374844713453       NaN  Merchant_85   
1   4008   2023-01-01 01:16:08  3460245159749480       NaN  Merchant_23   
2   1221   2023-01-01 01:24:28  7308701990157768     macOS  Merchant_70   
3   9609   2023-01-01 02:06:57  8454886440761098       X11  Merchant_33   
4   5689   2023-01-01 02:10:54  6350332939133843       NaN  Merchant_90   

      amt     trans_num   unix_time  is_fraud  first  ...      job  \
0  252.75  TRANS_662964  1672533543         0   Jane  ...      NaN   
1  340.17  TRANS_134939  1672535768         0  Alice  ...    Nurse   
2   76.38  TRANS_258923  1672536268         0    Bob  ...   Doctor   
3  368.88  TRANS_226814  1672538817         0   Mike  ...  Teacher   
4  323.32  TRANS_668449  1672539054         0   Mike  ...    Nurse   

          dob       category  merch_lat  merch_long merchant_id      lat  \
0  2002-10-12            NaN        NaN   76.433212 